This is just a little notebook that I am adding all of the functions I create. Hopefully, they will eventually be good enough to add to the real evaltools, but if they never get there than at least they will be useful to me. Obviously the single cell created here will have to be copied into a normal python script for this to work as an importable script. I am just using Jupyter as a convenient text editor.

In [2]:
""" Functions used to quickly graph evaluation plots for multiple stations and regions.
"""
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm


def byDepth(ax,df,obsvar,modvar,lims):
    ps=et.varvarPlot(ax,df,obsvar,modvar,'Z',(15,22),'z','m',('mediumseagreen','darkturquoise','navy'))
    l=ax.legend(handles=ps)
    ax.set_xlabel('Obs')
    ax.set_ylabel('Model')
    ax.plot(lims,lims,'k-',alpha=.5)
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_aspect(1)
    return ps,l

def byRegion(ax,df,datreg,obsvar,modvar,lims):
    ps=[]
    for ind, iregion in enumerate(df.Basin.unique()):
        ax.plot(datreg[iregion]['Lon'], datreg[iregion]['Lat'],'.',
                color = colors[ind], label=iregion)
        ps0=et.varvarPlot(ax,datreg[iregion],obsvar,modvar,
                        cols=(colors[ind],),lname=iregion)
        ps.append(ps0)
    l=ax.legend(handles=[ip[0][0] for ip in ps])
    ax.set_xlabel('Obs')
    ax.set_ylabel('Model')
    ax.plot(lims,lims,'k-',alpha=.5)
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_aspect(1)
    return ps,l

def byStation(ax,df,datstat,region,obsvar,modvar,lims):
    ps=[]
    for ind, istation in enumerate(df[df['Basin'] == region].Station.unique()):
        ax.plot(datstat[istation]['Lon'], datstat[istation]['Lat'],'.',
                    color = colors[ind], label=istation)
        ps0=et.varvarPlot(ax,datstat[istation],obsvar,modvar,
                        cols=(colors[ind],),lname=istation)
        ps.append(ps0)
    l=ax.legend(title='Stations',title_fontsize=20,handles=[ip[0][0] for ip in ps])
    ax.set_xlabel('Obs')
    ax.set_ylabel('Model')
    ax.plot(lims,lims,'k-',alpha=.5)
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_aspect(1)
    return ps,l

def bySeason(ax,seasons,obsvar,modvar,lims):
    for axi in ax:
        axi.plot(lims,lims,'k-')
        axi.set_xlim(lims)
        axi.set_ylim(lims)
        axi.set_aspect(1)
        axi.set_xlabel('Obs')
        axi.set_ylabel('Model')
    ps=et.varvarPlot(ax[0],seaons[0],obsvar,modvar,cols=('crimson','darkturquoise','navy'))
    ax[0].set_title('Jan-Mar')
    ps=et.varvarPlot(ax[1],seaons[1],obsvar,modvar,cols=('crimson','darkturquoise','navy'))
    ax[1].set_title('Apr')
    ps=et.varvarPlot(ax[2],seaons[2],obsvar,modvar,cols=('crimson','darkturquoise','navy'))
    ax[2].set_title('May-Aug')
    ps=et.varvarPlot(ax[3],seasons[3],obsvar,modvar,cols=('crimson','darkturquoise','navy'))
    ax[3].set_title('Sep-Dec')
    return 

def hist2d(ax,df,obsvar,modvar,lims,fontsize=12):
    ax.plot((-250,250),(-250,250),'k-',alpha=.2)
    ii=(~np.isnan(df[obsvar]))&(~np.isnan(df[modvar]))
    counts, xedges, yedges, ps=ax.hist2d(df.loc[ii,[obsvar]].values.flatten(),
                                      df.loc[ii,[modvar]].values.flatten(),bins=25*3,norm=LogNorm())
    cb=fig.colorbar(ps,ax=ax,label='Count',shrink=0.5)
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_ylabel('Modeled',fontsize=fontsize)
    ax.set_xlabel('Observed',fontsize=fontsize)
    plt.tight_layout()
    return ps  

def bySeason_hist2d(ax,seasons,obsvar,modvar,lims):
    for axj in ax:
        for axi in axj:
            axi.plot(lims,lims,'k-')
            axi.set_xlim(lims)
            axi.set_ylim(lims)
            axi.set_aspect(1)
            axi.set_xlabel('Obs')
            axi.set_ylabel('Model')
    jp=hist2d(seaons[0],ax[0][0],obsvar,modvar,lims)
    ax[0][0].set_title('Jan-Mar')
    jp=hist2d(seaons[1],ax[0][1],obsvar,modvar,lims)
    ax[0][1].set_title('Apr')
    jp=hist2d(seaons[2],ax[1][0],obsvar,modvar,lims)
    ax[1][0].set_title('May-Aug')
    jp=hist2d(seasons[3],ax[1][1],obsvar,modvar,lims)
    ax[1][1].set_title('Sep-Dec')
    return 

def ErrErr(df,fig,ax,obsvar1,modvar1,obsvar2,modvar2,lims1,lims2):
    m=ax.scatter(df[modvar1]-df[obsvar1],df[modvar2]-df[obsvar2],c=df['Z'],s=1,cmap='gnuplot')
    cb=fig.colorbar(m,ax=ax,label='Depth (m)')
    ax.set_xlim(lims1)
    ax.set_ylim(lims2)
    ax.set_aspect((lims1[1]-lims1[0])/(lims2[1]-lims2[0]))
    return m,cb

def multi_depreg_graph(df,datyear,years,obsvar,modvar,phyvar_name,lims,figsize):
    if type(years) == int:
        fig,ax=plt.subplots(1,2,figsize=figsize)
        ps,l=byDepth(ax[0],obsvar,modvar,lims,byyear=True,year=years)
        ax[0].set_title(f'{phyvar_name} (g kg$^-1$) By Depth for {years}')
        ps,l=byRegion(ax[1],obsvar,modvar,lims,year=years)
        ax[1].set_title(f'{phyvar_name} (g kg$^-1$) By Region for {years}');
    elif type(years) == list:
        fig,ax=plt.subplots(len(years),2,figsize=figsize)
        for d,Y in enumerate(years):
            ps,l=byDepth(ax[d][0],obsvar,modvar,lims,byyear=True,year=Y)
            ax[d][0].set_title(f'{phyvar_name} (g kg$^-1$) By Depth for {Y}')
            ps,l=byRegion(ax[d][1],obsvar,modvar,lims,year=Y)
            ax[d][1].set_title(f'{phyvar_name} (g kg$^-1$) By Region for {Y}');
    # put a raise exception thing into this. 
    plt.tight_layout()
        
# This has been altered but it has not been finished or tested !!!!!!!!!!!!!!!
def multi_enverr_graph(df,datyear,years,obsvar,modvar,envvar,envvar_name,figsize,):
    if type(years) == int:
        fig,ax=plt.subplots(1,len(envvar),figsize=figsize)
    elif type(years) == list:
        for d,Y in zip(len(years),years):
            for a,(o,m) in enumerate(zip(obsvar,modvar)):
                m=ax[d][a].scatter(datyear[Y][envvar],datyear[Y][obsvar]-datyear[Y][modvar],c=datyear[Y]['Z'],s=1,cmap='gnuplot') 
                cb=fig.colorbar(m,ax=ax[d][a],label='Depth (m)')
                ax[d][a].set_xlabel(f'Obs {envvar_name} (g kg$^{-1}$)',fontsize=12)
                ax[d][a].set_ylabel(f'{v[2]} Error ($\mu$M)',fontsize=12)
                ax[d][a].set_title(str(Y))
    # Put a raise exception thing into this. 
    plt.tight_layout()
        
def multi_timerror_graph(df,datyear,years,obsvar,modvar,down,figsize):
    fig,ax=plt.subplots(down,1,figsize=figsize)
    for d,Y in zip(range(down),years):
            m=ax[d].scatter(datyear[Y]['dtUTC'],datyear[Y][modvar]-datyear[Y][obsvar],s=8,cmap='gnuplot') 
            ax[d].set_xlabel(f'Date',fontsize=20)
            ax[d].set_ylabel(f'{obsvar} Error ($\mu$M)',fontsize=20)
            ax[d].set_title(str(Y), fontsize=22)
    plt.tight_layout()
        
def multi_station_graph(df,obsvar,modvar,regions,year,lims,down=6,figsize=(14,40)):
    """ A function that creates a series of scatter plots and maps for each region
        And shows the stations within each region colored on the graph and map. 
                    
    :arg obsvar,modvar: The name of the observed and model variables you wish to compare to each other.
    :type :string
    
    :are regions: The names of all of the basins you wish to look at 
    :type : list of strings
    
    :arg lims: A pair of values that will decide the range of the graph. Should always
                    should always be larger than the maximum value of the variable.
    :type : tuple
    
    :arg down: A number which should be equal to the number of regions you are looking at
    :type : integer
    
    :arg figsize: a pair of values that decide the size of the entire figure
    :type : tuple
    """
    fig, ax = plt.subplots(down,2,figsize = figsize)
    for d,r in zip(range(down),regions):
        ps=byStation(ax[d][0],obsvar,modvar,lims,r,year)
        ax[d][0].set_title(f'{obsvar} ($\mu$M) in {r} by Station');

        with nc.Dataset('/data/vdo/MEOPAR/NEMO-forcing/grid/bathymetry_201702.nc') as grid:
            viz_tools.plot_coastline(ax[d][1], grid, coords = 'map',isobath=.1)

        for ind, istation in enumerate(df[df['Basin'] == r].Station.unique()):
            ax[d][1].plot(datstat[istation]['Lon'], datstat[istation]['Lat'],'.',
                color = colors[ind], label=istation)
        ax[d][1].set_ylim(47, 49)
        ax[d][1].legend(bbox_to_anchor=[1,.6,0,0])
        ax[d][1].set_xlim(-124, -122);
        ax[d][1].set_title(f'Observation Locations for {r}');  

def logt(x):
    return np.log10(x+.001)

def multi_timerror_graph(df,datyear,years,obsvar,modvardown,figsize,):
    fig,ax=plt.subplots(down,1,figsize=figsize)
    for d,Y in zip(range(down),years):
            m=ax[d].scatter(datyear[Y]['dtUTC'],datyear[Y][modvar]-datyear[Y][obsvar],s=8,cmap='gnuplot') 
            ax[d].set_xlabel(f'Date',fontsize=20)
            ax[d].set_ylabel(f'{obsvar} Error ($\mu$M)',fontsize=20)
            ax[d].set_title(str(Y), fontsize=22)
            yearsFmt = mdates.DateFormatter('%d %b')
            ax[d].xaxis.set_major_formatter(yearsFmt)
    plt.tight_layout()
    
def multi_meanerr_graph(df,datyear,years,obsvar,modvar,down,figsize):
    fig,ax=plt.subplots(down,1,figsize=figsize)
    for d,Y in zip(range(down),years):
            meanerr=datyear[Y].groupby(by='dtUTC').mean()
            m=ax[d].plot(datyear[Y]['dtUTC'].unique(),meanerr[modvar]-meanerr[obsvar],'c-') 
            ax[d].set_xlabel(f'Date',fontsize=20)
            ax[d].set_ylabel(f'{obsvar} Error ($\mu$M)',fontsize=20)
            ax[d].set_title(str(Y), fontsize=22)
            yearsFmt = mdates.DateFormatter('%d %b')
            ax[d].xaxis.set_major_formatter(yearsFmt)
    plt.tight_layout()
    
def multi_timese_graph(df,datyear,years,obsvar,modvar,down,figsize):
    fig,ax=plt.subplots(down,1,figsize=figsize)
    for d,Y in zip(range(down),years):
            m=ax[d].scatter(datyear[Y]['dtUTC'],datyear[Y][obsvar],color='blue',s=8,cmap='gnuplot',label='Observed')
            s=ax[d].scatter(datyear[Y]['dtUTC'],datyear[Y][modvar],color='red',s=8,cmap='gnuplot',label='Model')
            ax[d].legend(loc='upper right')
            ax[d].set_xlabel(f'Date',fontsize=20)
            ax[d].set_ylabel(f'{obsvar} ($\mu$M)',fontsize=20)
            ax[d].set_title(str(Y), fontsize=22)
            yearsFmt = mdates.DateFormatter('%d %b')
            ax[d].xaxis.set_major_formatter(yearsFmt)
    plt.tight_layout()